ex01_imp

In [ ]:
import re # import modułu Regular Expressions
import requests  # import modułu do
url = "https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt" # wczytanie tekstu ze strony internetowej
response = requests.get(url)
response.encoding = 'utf-8'  # ustawienie kodowania
text = response.text

text = text.lower() # konwersja na małe litery

clear = "" # usunięcie znaków interpunkcyjnych
for mark in text:
    if mark not in [".","-","!","?"]:
        clear += mark

words = re.findall(r"\w+", clear, flags=re.UNICODE) # tokenizacja tekstu

unique_words = set() # inicjalizacja zbioru unikalnych słów

for word in words: # eliminacja duplikatów
    unique_words.add(word)

unique_list = list(unique_words) # konwersja zbioru na listę

n = len(unique_list) # sortowanie według długości i alfabetycznie [sortowanie bąbelkowe]
for i in range(n):
    for j in range(0, n-i-1):
        a = unique_list[j]
        b = unique_list[j+1]

        swap = False

        if len(a) < len(b):
            swap = True
        elif len(a) == len(b) and a > b:
            swap = True
        if swap:
            unique_list[j], unique_list[j+1] = unique_list[j+1], unique_list[j]

top10 = unique_list[:10] # prezentacja wyników końcowych
print("10 najdłuższych słów w »Panu Tadeuszu« (po usunięciu ., -, !, ?):")
for i in range(len(top10)):
    print(f"{i + 1}. {top10[i]} (długość: {len(top10[i])})")

ex01_func

In [ ]:
import re # import modułu Regular Expressions

PUNCT_TO_REMOVE = ".-!?,"  # zdefiniowanie stałej definiujacej znaki interpukcyjne do usunięcia

def read_text(url: str) -> str:  # pobieranie tekstu ze strony internetowej
    response = requests.get(url)
    response.encoding = 'utf-8'
    return response.text

def normalize(text: str) -> str: # zamiana na małe litery i usunięcie wybranych znaków interpunkcyjnych
    text_lower = text.lower()
    text_clean = text_lower.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    return text_clean

def to_words(text: str) -> list[str]: # zamiana tekstu na listę słów
    return re.findall(r"\w+", text, flags=re.UNICODE)

def unique(words: list[str]) -> list[str]: # utworzenie listy unikalnych słów
    return list(set(words))

def sort_words_by_length(words: list[str], n: int = 10) -> list[str]: # zwracanie n najdłuższych słów, przy remisie sortując alfabetycznie
    return sorted(words, key=lambda w: (-len(w), w))[:n]

def compose(*functions): # złożenie funkcji [1. wczytaj_tekst → 2. normalizuj_tekst → 3. zamień_na_słowa → 4. unikalne_słowa → 5. sortuj_słowa_po_długosci]
    def inner(arg):
        result = arg
        for func in reversed(functions):
            result = func(result)
        return result
    return inner

def pipeline(data, *functions): # etap sekwencyjnego przetwrzania
    result = data
    for func in functions:
        result = func(result)
    return result

def format_results(words: list[str]) -> str: # format wyników do czytelnej postaci
    lines = ["10 najdłuższych słów w »Panu Tadeuszu« (po usunięciu ., -, !, ?, ,):"]
    for i, w in enumerate(words, start=1):
        lines.append(f"{i}. {w} (długość: {len(w)})")
    return "\n".join(lines)


def main_functional_compose():
    process_text = compose(
        lambda words: sort_words_by_length(words, n=10),
        unique,
        to_words,
        normalize,
        read_text
    )
    top10 = process_text("https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt")
    print(format_results(top10))

if __name__ == "__main__": # wykonanie głównej funkcji
    main_functional_compose()

ex02_imp

In [ ]:
def read_text(url: str) -> str: # definicja funkcji odczytu tekstu ze strony internetowej
    response = requests.get(url)
    response.encoding = 'utf-8'
    return response.text

def find_most_common_words_imperative(text: str, n: int = 10, min_length: int = 6) -> list[tuple[str, int]]: # definicja funkcji znajdującej najczęściej występującej słowa w tekście

    text_lower = text.lower() # konwersja na małe litery

    punct_to_remove = ".-!?,"  # filtrowanie znaków interpunkcyjnych
    cleaned_text = ""
    for char in text_lower:
        if char not in punct_to_remove:
            cleaned_text += char

    all_words = []  # tokenizacja tekstu
    current_word = ""
    for char in cleaned_text:
        if char == " " or char == "\n" or char == "\t":
            if current_word != "":
                all_words.append(current_word)
                current_word = ""
        else:
            current_word += char
    if current_word != "":
        all_words.append(current_word)

    words = []  # filtrowanie krótkich słów
    for word in all_words:
        if len(word) > min_length:
            words.append(word)

    word_counts = {} #  # zliczanie wystąpień słów
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    word_list = [] # konwersja słownika na listę krotek
    for word in word_counts:
        word_list.append((word, word_counts[word]))


    for i in range(len(word_list)): # sortowanie malejąco według liczby wystąpień (bubble sort)
        for j in range(i + 1, len(word_list)):
            if word_list[j][1] > word_list[i][1]:
                temp = word_list[i]
                word_list[i] = word_list[j]
                word_list[j] = temp


    result = [] # zwrócenie top n wyników
    limit = n
    if len(word_list) < n:
        limit = len(word_list)

    for i in range(limit):
        result.append(word_list[i])

    return result

def main(): # definicja funkcji głównej
    print("Analiza najczęstszych słów w »Panu Tadeuszu«")
    print("(po usunięciu znaków: . - ! ? ,)")
    print("(tylko słowa dłuższe niż 6 znaki)\n")

    text = read_text("https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt")

    top_10 = find_most_common_words_imperative(text, n=10, min_length=6)

    print("10 najczęstszych słów:")
    print("-" * 40)

    for i in range(len(top_10)):
        word, count = top_10[i]
        print(f"{i + 1:2d}. {word:15s} - {count:4d} wystąpień")


if __name__ == "__main__": # wykonanie funkcji głównej
    main()

ex02_func

In [ ]:
import requests
from collections import Counter

def read_text(url: str) -> str: # definicja funkcji read text
    response = requests.get(url)
    response.encoding = 'utf-8'
    return response.text

def find_most_common_words_functional(text: str, n: int = 10, min_length: int = 6) -> list[tuple[str, int]]: # definicja funkcji find_most_common_words_functional

    punct_to_remove = ".-!?," # deklaracja zmiennej punct_to_remove

    return Counter( # przetwarzanie tekstu i zwracanie najczęstszych słów
        filter(
            lambda word: len(word) > min_length,
            "".join(
                map(lambda char: "" if char in punct_to_remove else char, text.lower())
            ).split()
        )
    ).most_common(n)


def main(): # główna funkcja
    print("Analiza najczęstszych słów w »Panu Tadeuszu«")
    print("(po usunięciu znaków: . - ! ? ,)")
    print("(tylko słowa dłuższe niż 6 znaki)\n")

    text = read_text("https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt")

    top_10 = find_most_common_words_functional(text, n=10, min_length=6)

    print("10 najczęstszych słów:")
    print("-" * 40)

    for i, (word, count) in enumerate(top_10, 1):
        print(f"{i:2d}. {word:15s} - {count:4d} wystąpień")


if __name__ == "__main__": # wykonanie głównej funkcji
    main()

ex03_imp

In [ ]:
def calculate_word_length_histogram_imperative(text: str, max_length: int = 10) -> dict[int, int]: # definicja funkcji wyznaczającej długość słów w tekscie

    punct_to_remove = ".-!?," # usunięcie znaków interpunkcyjnych
    cleaned_text = ""

    for char in text.lower():
        if char not in punct_to_remove:
            cleaned_text += char
        else:
            cleaned_text += " "


    words = cleaned_text.split() # podział na słowa


    histogram = {}    # zliczanie długości słów do max długości

    for word in words:
        length = len(word)
        if 0 < length <= max_length:
            if length in histogram:
                histogram[length] += 1
            else:
                histogram[length] = 1

    return histogram


def read_text(url: str) -> str: # wczytanie tekstu ze strony internetowej
    response = requests.get(url)
    response.encoding = 'utf-8'
    return response.text


def print_histogram_horizontal(histogram: dict[int, int], max_length: int = 10) -> None: # zdefiniowanie funkcji histogram

    print("Histogram długości słów w »Panu Tadeuszu«")
    print("(po usunięciu znaków: . - ! ? ,)")
    print("(słowa o długości 1-10 liter)")
    print("=" * 80)

    if not histogram:
        print("Brak danych do wyświetlenia")
        return

    max_count = max(histogram.values()) # zanlezienie maksymalnej liczby wystąpień

    chart_height = 20 #wysokość wykresu (liczba wierszy)

    scale = max_count / chart_height if max_count > 0 else 1 # obliczanie skali

    for row in range(chart_height, 0, -1): # rozpoczęcie rysowania wykresu od gory do dołu

        threshold = row * scale # wartość progowa dla wiersza

        print(f"{int(threshold):5d} |", end="") # wyświetlenie wartości na osi Y

        for length in range(1, max_length + 1): # dla każdej długości słowa (1-10)
            count = histogram.get(length, 0)

            if count >= threshold: # warunek liczba wystąpień przekracza próg, rysuj słupek
                print("  ███", end="")
            else:
                print("     ", end="")

        print()  # nowa linia

    print("      " + "─" * (max_length * 5)) # oś X (linia pozioma)

    print("      ", end="")  # etykieta osi X (długości słów)
    for length in range(1, max_length + 1):
        print(f"  {length:2d} ", end="")
    print("\n")

    print("Dokładne wartości:") # legenda z dokładnymi wartościami
    print("-" * 80)
    for length in range(1, max_length + 1):
        count = histogram.get(length, 0)
        print(f"Długość {length:2d}: {count:6d} wystąpień")


def main(): # funkcja główna
    max_length = 10

    text = read_text("https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt") # wczytanie tekstu ze strony

    histogram = calculate_word_length_histogram_imperative(text, max_length) # wczytanie histogramu

    print_histogram_horizontal(histogram, max_length) # wyświetlenie wyników (histogram poziomy)

    print("\n" + "=" * 80) # dodatkowe statystyki
    print("Statystyki (dla słów 1-10 liter):")
    total_words = sum(histogram.values())
    print(f"Łączna liczba słów (1-10 liter): {total_words}")

    if histogram: # najczęstsza długość słowa

        most_common_length = max(histogram.keys(), key=lambda k: histogram[k])
        print(f"Najczęstsza długość słowa: {most_common_length} ({histogram[most_common_length]} wystąpień)")


if __name__ == "__main__": # wykonanie funkcji głównej
    main()

ex03_func

In [ ]:
from collections import Counter # importuje klasę counter z modułu collections

def read_text(url: str) -> str: #  funkcja wczytania tekstu ze strony internetowej
    response = requests.get(url)
    response.encoding = 'utf-8'
    return response.text

def calculate_word_length_histogram_functional(text: str, max_length: int = 10) -> dict[int, int]: # wyznaczenie histogramu długości słów w tekście

    punct_to_remove = ".-!?," # blok kodu mający na celu usunięcie interpunkcji, podział,filtracje i zliczanie

    return dict(
        Counter(
            map(
                len,  # mapowanie słów na ich długości
                filter(
                    lambda word: 0 < len(word) <= max_length,  # filtracja długości
                    "".join(
                        map(
                            lambda char: " " if char in punct_to_remove else char,
                            text.lower()
                        )
                    ).split()
                )
            )
        )
    )


def print_histogram_horizontal(histogram: dict[int, int], max_length: int = 10) -> None: # wyświetlenie histogramu w formie poziomej
    print("Histogram długości słów w »Panu Tadeuszu«")
    print("(po usunięciu znaków: . - ! ? ,)")
    print("(słowa o długości 1-10 liter)")
    print("(podejście funkcyjne)")
    print("=" * 80)

    if not histogram:
        print("Brak danych do wyświetlenia")
        return

    max_count = max(histogram.values()) if histogram else 0 # znalezienie maksymalnej liczby wystąpień

    chart_height = 20 # wysokość wykresu (liczba wierszy)

    scale = max_count / chart_height if max_count > 0 else 1 # obliczanie skali histogramu

    rows = range(chart_height, 0, -1) # generowanie wierszy wykresu

    for row in rows:
        threshold = row * scale
        print(f"{int(threshold):5d} |", end="")

        # funkcyjne generowanie słupków dla każdej długości
        bars = map(
            lambda length: "  ███" if histogram.get(length, 0) >= threshold else "     ",
            range(1, max_length + 1)
        )

        print("".join(bars))

    print("      " + "─" * (max_length * 5)) # oś X

    print("      " + "".join(map(lambda l: f"  {l:2d} ", range(1, max_length + 1))))
    print() # etykiety osi X

    print("Dokładne wartości:") # legenda z dokładnymi wartościami
    print("-" * 80)

    list(map(
        lambda length: print(f"Długość {length:2d}: {histogram.get(length, 0):6d} wystąpień"),
        range(1, max_length + 1) # wyświetlenie wartości
    ))


def main():
    max_length = 10

    # wczytanie tekstu
    text = read_text("https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt")

    # wyznaczenie histogramu
    histogram = calculate_word_length_histogram_functional(text, max_length)

    # wyświetlenie wyników
    print_histogram_horizontal(histogram, max_length)

    # dodatkowe statystyki
    print("\n" + "=" * 80)
    print("Statystyki (dla słów 1-10 liter):")

    total_words = sum(histogram.values())
    print(f"Łączna liczba słów (1-10 liter): {total_words}")

    if histogram:
        most_common_length = max(histogram.keys(), key=lambda k: histogram[k])
        print(f"Najczęstsza długość słowa: {most_common_length} ({histogram[most_common_length]} wystąpień)")


if __name__ == "__main__": # # wykonanie funkcji głównej
    main()